# Control Cinemático PR

Seguimiento de trayectorias circulares para un manipulador con una articulación prismática seguida de una articulación rotacional.

## 1. Configuración del experimento
Definimos la duración de la simulación, el radio de la trayectoria y los parámetros geométricos del robot.

In [ ]:
import sys
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

plt.rcParams['animation.embed_limit'] = 60

ROOT = pathlib.Path().resolve().parents[1]
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from notebooks.control.utils import time_vector, circular_task_reference, plot_tracking

In [ ]:
sim_config = {"t0": 0.0, "tf": 20.0, "n_steps": 600}
radius = 0.12
center = (0.25, 0.18)
omega = 2 * np.pi / (sim_config["tf"] - sim_config["t0"])

t, dt = time_vector(sim_config)
Xd, Vd = circular_task_reference(radius, center, omega, t)

L2 = 0.25

## 2. Modelo cinemático
Funciones para el cálculo de la cinemática directa y la matriz Jacobiana del manipulador PR.

In [ ]:
def forward_kinematics(q: np.ndarray) -> np.ndarray:
    q1, q2 = q
    x = q1 + L2 * np.cos(q2)
    y = L2 * np.sin(q2)
    return np.array([x, y])


def jacobian(q: np.ndarray) -> np.ndarray:
    _, q2 = q
    return np.array(
        [
            [1.0, -L2 * np.sin(q2)],
            [0.0, L2 * np.cos(q2)],
        ]
    )

## 3. Control cinemático
Aplicamos un controlador proporcional en el espacio cartesiano y resolvemos las velocidades articulares con la pseudo-inversa del Jacobiano.

In [ ]:
Kp = np.diag([6.0, 6.0])
q = np.zeros((2, t.size))
q[:, 0] = np.array([center[0], np.pi / 6])
X = np.zeros((2, t.size))

for k in range(t.size - 1):
    X[:, k] = forward_kinematics(q[:, k])
    error = Xd[:, k] - X[:, k]
    v_ref = Vd[:, k] + Kp @ error
    J = jacobian(q[:, k])
    dq = np.linalg.pinv(J) @ v_ref
    q[:, k + 1] = q[:, k] + dq * dt

X[:, -1] = forward_kinematics(q[:, -1])

## 4. Verificación del error
Comprobamos que el error máximo de seguimiento se mantiene dentro de un umbral tolerable.

In [ ]:
tracking_error = np.linalg.norm(Xd - X, axis=0)
max_error = float(tracking_error.max())
print(f"Error máximo de seguimiento: {max_error:.4f} m")
assert max_error < 0.02, "El error de seguimiento excede el umbral de 2 cm"

## 5. Trayectoria y errores

In [ ]:
fig = plot_tracking(X, Xd, t)
fig

## 6. Animación del movimiento
Se visualiza el desplazamiento del efector final sobre la trayectoria circular.

In [ ]:
def arm_points(q: np.ndarray) -> np.ndarray:
    q1, q2 = q
    base = np.array([0.0, 0.0])
    joint = np.array([q1, 0.0])
    end = joint + L2 * np.array([np.cos(q2), np.sin(q2)])
    return np.stack((base, joint, end), axis=1)


fig, ax = plt.subplots(figsize=(5, 5))
ax.set_aspect("equal", adjustable="box")
ax.set_xlim(0.0, 0.6)
ax.set_ylim(-0.2, 0.6)
ax.set_xlabel("x [m]")
ax.set_ylabel("y [m]")
ax.set_title("Control cinemático PR")
ax.plot(Xd[0], Xd[1], "--", color="tab:orange", label="Trayectoria deseada")
arm_line, = ax.plot([], [], "o-", lw=2, color="tab:blue", label="Robot")
ax.legend(loc="upper right")

frames = range(0, t.size, 2)


def init():
    arm_line.set_data([], [])
    return (arm_line,)


def animate(i):
    pts = arm_points(q[:, i])
    arm_line.set_data(pts[0], pts[1])
    return (arm_line,)


anim = animation.FuncAnimation(
    fig, animate, init_func=init, frames=frames, interval=50, blit=True
)
HTML(anim.to_jshtml())